# Create Slice for Fabric Rack
This is the slice creation for the ```OpenVS Flow``` tutorial. This is the first step of the assignment, to create the slice on fabric using fablib.
 

## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 4 nodes for the OSPF Assignment.
### 1.1 Reserve Resources
In this section we will declare the structure of the slice we will be using.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "OpenVSSwitch"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

In [ ]:
# Setup Network
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Bridge/switch node
    OVSbr = slice.add_node(name="Bridge", site=site)
    OVSbr.set_image('default_ubuntu_20')
    AP1 = OVSbr.add_component(model='NIC_Basic', name="CPort1").get_interfaces()[0] 
    AP2 = OVSbr.add_component(model='NIC_Basic', name="H1Port").get_interfaces()[0]
    AP3 = OVSbr.add_component(model='NIC_Basic', name="H2Port").get_interfaces()[0]
    AP4 = OVSbr.add_component(model='NIC_Basic', name="H3Port").get_interfaces()[0]
    
    #H1
    Host_1 = slice.add_node(name="Host_1", site=site)
    Host_1.set_image('default_ubuntu_20')
    BP1 = Host_1.add_component(model=nicmodel, name="BPort1").get_interfaces()[0] 
    
    #H2
    Host_2= slice.add_node(name="Host_2", site=site)
    Host_2.set_image('default_ubuntu_20')
    CP1 = Host_2.add_component(model=nicmodel, name="CPort1").get_interfaces()[0] 
    
    #H3
    Host_3 = slice.add_node(name="Host_3", site=site)
    Host_3.set_image('default_ubuntu_20')
    DP1 = Host_3.add_component(model=nicmodel, name="DPort1").get_interfaces()[0]    
    
    #Connection "internet". remove connection later
    Ctrl = slice.add_node(name="ControllerNode", site=site)
    Ctrl.set_image('default_ubuntu_20')
    EP1 = Ctrl.add_component(model=nicmodel, name="DPort1").get_interfaces()[0]
    
    Lan1 = slice.add_l2network(name="ControllerLan", interfaces=[EP1, AP1])
    Lan2 = slice.add_l2network(name="H1Lan", interfaces=[BP1, AP2])
    Lan3 = slice.add_l2network(name="H2Lan", interfaces=[CP1, AP3])
    Lan4 = slice.add_l2network(name="H3Lan", interfaces=[DP1, AP4])
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
In this section of the slice creation, once the nodes are active, will assign the network spaces some predefined values that we will use for the assignment

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    OVSbr = slice.get_node(name="Bridge") 
    Host_1 = slice.get_node(name="Host_1")
    Host_2 = slice.get_node(name="Host_2")
    Host_3 = slice.get_node(name="Host_3")
    Ctrl = slice.get_node(name="ControllerNode")
    
    subnet1 = IPv4Network("10.10.1.0/24")
    subnet2 = IPv4Network("10.10.2.0/24")
    
    AP1 = OVSbr.get_interface(network_name="ControllerLan") 
    AP1.ip_addr_add(addr="10.10.2.1", subnet=subnet2)
    AP2 = OVSbr.get_interface(network_name="H1Lan") 
    AP2.ip_addr_add(addr="10.10.1.11", subnet=subnet1)
    AP3 = OVSbr.get_interface(network_name="H2Lan") 
    AP3.ip_addr_add(addr="10.10.1.12", subnet=subnet1)
    AP4 = OVSbr.get_interface(network_name="H3Lan") 
    AP4.ip_addr_add(addr="10.10.1.13", subnet=subnet1)
    
    BP1 = Host_1.get_interface(network_name="H1Lan") 
    BP1.ip_addr_add(addr="10.10.1.1", subnet=subnet1)
         
    CP1 = Host_2.get_interface(network_name="H2Lan") 
    CP1.ip_addr_add(addr="10.10.1.2", subnet=subnet1)
    
    DP1 = Host_3.get_interface(network_name="H3Lan") 
    DP1.ip_addr_add(addr="10.10.1.3", subnet=subnet1)
    
    CtrlP1 = Ctrl.get_interface(network_name="ControllerLan") 
    CtrlP1.ip_addr_add(addr="10.10.2.2", subnet=subnet2)
    
    for node in slice.get_nodes():
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
        
except Exception as e:
    print(f"Exception: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
This section of the slice creation, we will add any aditional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software
try:
    for node in slice.get_nodes():
        node.execute('yes | sudo apt-get -y update && sudo apt-get upgrade -y') 
        node.execute('sudo apt-get -y install net-tools')
        
    OVSbr.execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common')
    Ctrl.execute('sudo apt install ppython')
    Ctrl.execute('sudo apt install python3-pip')
    Ctrl.execute('sudo apt install gcc python-dev libffi-dev libssl-dev libxml2-dev libxslt1-dev zlib1g-dev -y')
    Ctrl.execute('sudo apt-get install python3-ryu -y')
    Ctrl.execute('git clone https://github.com/faucetsdn/ryu.git')


except Exception as e:
    print(f"Fail: {e}")

    

## Continue to The experiment notebook

Once you have completed this notebook you should be able to continue to the OSPF Notebook. You can either open it on the explorer or click [Here](./OSPF.ipynb) to open the next notebook.